# Clasificación de Flores Iris

Alan Badillo Salas (badillo.soft@hotmail.com)

Vamos a clasificar las flores iris basados en sus características extraídas como el ancho y alto del sépalo, el ancho y alto del pétalo. Para hacerlo usaremos el dataset de UCI [Machine Learning Datasets](https://archive.ics.uci.edu/ml/), en específico [Dataset Iris](https://archive.ics.uci.edu/ml/datasets/Iris).

Las flores iris se clasifican en las familas `Vertiginosa`, `Versicolor` y `Virginica`.

## Paso 1 - Adquirir los datos para la clasificación

Vamos a extraer los datos desde https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

Usaremos `pd.read_csv(...)` de pandas para extrar un *DataFrame* el cuál contenga las filas y columnas de datos con la información de nuestras muestras de entrenamiento de la flor iris.

In [5]:
import pandas as pd

# La url desde dónde se van extraer los datos
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
## url = "iris.data"

# Definimos las columnas que serán utilizadas en el dataframe para entender los datos
columns = [ "Sepal-Length", "Sepal-Width", "Petal-Length", "Petal-Width", "Class" ]

# Leemos los datos indicando que los datos no traen cabeceras que describan cada columna
data = pd.read_csv(url, header=None, names=columns)

# Mostramos los primeros 5 datos
data.head()

,Sepal-Length,Sepal-Width,Petal-Length,Petal-Width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Paso 2 - Generamos la columna `Class-Cat` desde la columna `Class`

Tomaremos los datos de la columna `Class` para convertir las categorías en números y poder manipular los datos eb un vector.

Lo primero es determinar cuáles son las distintas categorías que aparecen en la columna `Class`.

In [7]:
# Mostramos el conjunto de valores distintos en la columna `Class`
set(data["Class"])

{'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}

Establecemos la columna `Class-Cat` a partir de la columna `Class`, dónde los valores serán el resultado de mapear los datos `{'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}` a números enteros.

In [9]:
# Creamos la columna `Class-Cat` como el mapeo de la columna `Class`
# dónde el valor "Iris-setosa" se va a sustituir por 1
# dónde el valor "Iris-versicolor" se va a sustituir por 2
# dónde el valor "Iris-virginica" se va a sustituir por 3
data["Class-Cat"] = data["Class"].map({
    "Iris-setosa": 1,
    "Iris-versicolor": 2,
    "Iris-virginica": 3,
})

# Mostramos 5 datos aleatorios en nuestro dataframe
data.sample(5)

,Sepal-Length,Sepal-Width,Petal-Length,Petal-Width,Class,Class-Cat
107,7.3,2.9,6.3,1.8,Iris-virginica,3
80,5.5,2.4,3.8,1.1,Iris-versicolor,2
98,5.1,2.5,3.0,1.1,Iris-versicolor,2
11,4.8,3.4,1.6,0.2,Iris-setosa,1
0,5.1,3.5,1.4,0.2,Iris-setosa,1


## Paso 3* - Aleatorizar los datos

Antes de extraer cualquier dato del dataframe, debemos garantizar que estos estén distribuidos aleatoriamente, para que cuándo partamos los datos en `X_train`, `Y_train`, `X_test` y `Y_test` al `80%` y `20%` estos sean aleatorio pero diferentes (no se repetirá una muestra de `train` en `test`).

In [10]:
# Tomanos un ejemplo aleatorio de todas las muestras
data = data.sample(len(data))

data.head()

,Sepal-Length,Sepal-Width,Petal-Length,Petal-Width,Class,Class-Cat
10,5.4,3.7,1.5,0.2,Iris-setosa,1
33,5.5,4.2,1.4,0.2,Iris-setosa,1
63,6.1,2.9,4.7,1.4,Iris-versicolor,2
88,5.6,3.0,4.1,1.3,Iris-versicolor,2
31,5.4,3.4,1.5,0.4,Iris-setosa,1


## Paso 4 - Partir las muestras en `Muestras de Entrenamiento` y `Muestras de Prueba`

Antes de poder clasificar los datos debemos obtener el 80% de las muestras (quizás 90%) si se desea y construir la matriz (el subdataframe) que tenga las muestras para `X` y el vetor (la columna) que tenga las muestras para `Y`.

Formamos `X_train` a partir de las columnas `Sepal-Length, Sepal-Width, Petal-Length, Petal-Width` tomando las primeras 120 muestras (80% de los datos).

In [13]:
# Número total de muestras en nuestro dataframe
N = len(data)

# L - Es un entero que nos dice cuántas muestras tomar para el 80% de los datos
L = int(0.8 * N)

print("Se van a tomar {} de un total de {} ({:.2f}%) para hacer el entrenamiento".format(L, N, 100 * L / N))

# Formamos el sub-dataframe `X_train` sólo con el 80% y las columnas deseadas para el entrenamiento
X_train = pd.DataFrame(data[0:L], columns=["Sepal-Length", "Sepal-Width", "Petal-Length", "Petal-Width"])

X_train.head()

Se van a tomar 120 de un total de 150 (80.00%)


,Sepal-Length,Sepal-Width,Petal-Length,Petal-Width
10,5.4,3.7,1.5,0.2
33,5.5,4.2,1.4,0.2
63,6.1,2.9,4.7,1.4
88,5.6,3.0,4.1,1.3
31,5.4,3.4,1.5,0.4


Para formar `Y_train` debemos recordar usar un vector (serie) en lugar de formar una matriz (sub-dataframe)

In [14]:
# Formamos el vector (la Serie) `Y_train` con los primero `L` valores de la columna `Class-Cat`
Y_train = data["Class-Cat"][0:L]

# Mostramos los primeros 5 valores del vector
Y_train[0:5]

10    1
33    1
63    2
88    2
31    1
Name: Class-Cat, dtype: int64

## Paso 5 - Crear un clasificador sobre `X_train` y `Y_train`

Podemos proponer un clasificador `SVC`, `MLPClassifier`, `SGDClassifier`, etc.

La idea es crear el clasificador y ajustarlo a las muestras de entrenamiento X_train` y `Y_train`.

In [15]:
from sklearn.svm import SVC

clf = SVC()

clf.fit(X_train, Y_train)

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

Una vez ajustado (entrenado) el clasificador ya puede predecir muestras

In [16]:
clf.predict([
    [1, 2, 1.5, 2.4], # Muestra de prueba 1
    [5, 3, 6, 1],     # Muestra de prueba 2
])

array([3, 3], dtype=int64)

## Paso 6 - Utilizar el 20% restante de muestras para ver que tan bien entranado quedó el clasificador

Vamos a validar cuántos aciertos/errores tiene nuestro clasificador sobre el 20% de las muestras reales con las que no se entrenó (que desconoce).

Lo primero será generar la matrix (sub-dataframe) de prueba `X_test` y el vector (serie) de prueba `Y_test`.

In [20]:
print("Se van a tomar {} de un total de {} ({:.2f}%) para hacer la validación".format(N - L, N, 100 * (N - L) / N))

# Construimos `X_test` mediante las restantes `L-N` muestras, es decir, del índice `L` al último índice `N-1`
X_test = pd.DataFrame(data[L:N], columns=["Sepal-Length", "Sepal-Width", "Petal-Length", "Petal-Width"])

X_test.head()

Se van a tomar 30 de un total de 150 (20.00%) para hacer la validación


,Sepal-Length,Sepal-Width,Petal-Length,Petal-Width
56,6.3,3.3,4.7,1.6
127,6.1,3.0,4.9,1.8
42,4.4,3.2,1.3,0.2
96,5.7,2.9,4.2,1.3
41,4.5,2.3,1.3,0.3


In [21]:
# Formamos el vector (serie) `Y_test` con los restantes `N-L` datos de la columna `Class-Cat`
Y_test = data["Class-Cat"][L:N]

Y_test[0:5]

56     2
127    3
42     1
96     2
41     1
Name: Class-Cat, dtype: int64

> Para hacer las pruebas de validación, por cada ejemplo de prueba deberemos generar una prediccón, la cuál podremos comparar contra los datos reales. Es decir, Dado `X_test` predecimos `Y_predict` y lo comparamos con `Y_test`, contando cuántos `Y_predict` son iguales/distintos a los `Y_test`.

In [25]:
Y_predict = clf.predict(X_test)

K = len(Y_test)

# Creamos un vector de diferencias absolutas, el cuál contenga 0 si no hay diferencia
# entre la i-ésima predicción y la i-ésima prueba
# para cada i-prueba
diferencias = [ abs(Y_predict[i] - Y_test.iloc[i]) for i in range(K) ]

incorrectos = sum(diferencias)
correctos = K - incorrectos

error = 100 * incorrectos / K
bien = 100 - error

print("Fallaron {} de {} pruebas ({}% error / {}% correcto)".format(incorrectos, K, error, bien))

Fallaron 1 de 30 pruebas (3% error / 97% correcto)
